In [1]:
import os
from glob import glob
from pymystem3 import Mystem
import pandas as pd
import json
import pickle

In [2]:
pp = os.getcwd()
PATH_DD = pp.split('group_mlada')[0] + 'group_mlada/'

In [3]:
train_path = f'{PATH_DD}/data/train/train.parquet'
train = pd.read_parquet(train_path, engine='pyarrow')

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1168516 entries, 0 to 1168515
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype
---  ------      --------------    -----
 0   variantid1  1168516 non-null  int64
 1   variantid2  1168516 non-null  int64
 2   target      1168516 non-null  int64
dtypes: int64(3)
memory usage: 26.7 MB


In [5]:
train.head(10)

,variantid1,variantid2,target
0,1447875869,1447872068,1
1,1176231201,284733670,1
2,658617865,549848659,0
3,719320625,719370486,1
4,1067645658,949954740,0
5,504121134,304997246,0
6,690909146,1044397832,1
7,691943502,691868704,0
8,479881093,256110461,1
9,905617449,905617985,0


In [8]:
attributes_path = f'{PATH_DD}/data/train/attributes.parquet'
attributes = pd.read_parquet(attributes_path, engine='pyarrow')

In [9]:
attributes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2252569 entries, 0 to 2252568
Data columns (total 3 columns):
 #   Column                             Dtype 
---  ------                             ----- 
 0   variantid                          int64 
 1   categories                         object
 2   characteristic_attributes_mapping  object
dtypes: int64(1), object(2)
memory usage: 51.6+ MB


In [10]:
attributes.head(10)

,variantid,categories,characteristic_attributes_mapping
0,47920382,"{""1"": ""EPG"", ""2"": ""Детские товары"", ""3"": ""Игру...","{""Цвет товара"": [""бежевый"", ""светло-розовый""],..."
1,49801845,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Количество в упаковке, шт"": [""1""], ""Бренд"": ..."
2,49853444,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Бренд"": [""Vervaco""], ""Тип"": [""Набор для выши..."
3,49893028,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Цвет товара"": [""серый""], ""Ширина, см"": [""0.8..."
4,49987483,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Цвет товара"": [""разноцветный""], ""Название цв..."
5,49995691,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Тип"": [""Настольная игра""], ""Страна-изготовит..."
6,50101275,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Цвет товара"": [""зеленый"", ""прозрачный""], ""На..."
7,50373843,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Цвет товара"": [""синий""], ""Количество в упако..."
8,51158084,"{""1"": ""EPG"", ""2"": ""Дом и сад"", ""3"": ""Подушки, ...","{""Артикул"": [""ОШО 4 - 3 Л""], ""Тип"": [""Одеяло""]..."
9,51252362,"{""1"": ""EPG"", ""2"": ""Дом и сад"", ""3"": ""Столовая ...","{""Артикул"": [""991""], ""Комплектация"": [""- чайни..."


In [11]:
print(len(attributes))

2252569


#### Составление описания объекта
<br>
Категории<br>

Начинаем со второй <br>
категория 2 под литерой a + номер от 0 до 23<br>
категория 3 под литерой b + номер от 0 до 582<br>
категория 4 под литерой c + номер от 0 до 3685<br>
<br>
характеристики под литерой d + номер от 0 до 6015<br><br>
Пример описания объекта<br>

a18 b320 c1643 d12 d67 d2845 

На первом  этапе обучение на описании только кодами ктегорий и характкристик <br>
На втором  этапе обучение на первом описании + все слова из характеристик<br>
На третьем этапе обучение на первом описании + все слова из характеристик + все цифры из характеристик<br>



In [14]:
def get_dict(name_f:str) ->dict:
    '''
    Получение словаря из файла
    '''
    r_d= {}
    with open(PATH_DD + name_f, 'r') as f:
        for line in f:
            line_l = line.strip().split(';')
            r_d[line_l[0]] = line_l[2]
    return r_d

In [15]:
# категории как признаки
d_characteristic = get_dict('data/characteristic.csv')
d_categories2 = get_dict('data/stat_categories2.csv')
d_categories3 = get_dict('data/stat_categories3.csv')
d_categories4 = get_dict('data/stat_categories4.csv')
d_all = {'d2': d_categories2,
         'd3': d_categories3,
         'd4': d_categories4,
         'dc': d_characteristic} 

In [56]:
def print_obj(desc_all_obj, nn: int):
    # печать объекта из словаря объектов
    k = 0
    for key in desc_all_obj:
        if k == nn:
                print(key, desc_all_obj[key], sep=' ==> ')
                break
        k += 1

    

In [57]:
# получение всех объектов
desc_all_obj ={}
with open(PATH_DD + 'data/all_obj.csv', 'r') as f:
    for line in f:
        line_l = line.strip().split(';')
        desc_all_obj[line_l[0].strip()] = line_l[1]
print(len(desc_all_obj))

2252569


In [58]:
print_obj(desc_all_obj, 999)
print_obj(desc_all_obj, 1000)
print_obj(desc_all_obj, 1001)

628056916 ==> b9 c52 d691 e978 e494 e305 e423 e239 e230 e297 e580 e257 e19 e9 e518 e4 e14 e255 e0 e1 e452 e298 e405 e380 e68 e700 e394 e463 e638 e241 e363 e236 e359 e419 e849 e412 e672 e268 e783 e244 e273 e289 e238 e252 e616 e2343 e2 e418 e443 e685 e403 e3 e6 e275 e69 e295
628105125 ==> b4 c252 d1757 e4 e14 e100 e27 e0 e1 e63 e44 e182 e1780 e3 e1799 e6 e151 e2 e145 e5
628139361 ==> b4 c2 d583 e8 e27 e0 e2


In [18]:
# Получение массива для обучения описанного в метках категорий
label = []
X=[]
Y=[]
for row in train.iterrows():
    # print(row)
    # print(row[1]['variantid1'])
    # print(row[1]['variantid2'])
    # print(row[1]['target'])
    label.append(row[0])
    if str(row[1]['variantid1']).strip() in desc_all_obj and str(row[1]['variantid2']).strip() in desc_all_obj:
        # print(desc_all_obj[row[1]['variantid1']])
        X.append(
            desc_all_obj[str(row[1]['variantid1']).strip()] + ' ' +
            desc_all_obj[str(row[1]['variantid2']).strip()])
        Y.append(row[1]['target'])
    # else:
    #     print('not found')
    #break
print(len(X))

1168516


In [19]:
X[100]

'b10 c251 d747 e39 e18 e0 e1 e7 e20 e56 e16 e3 e6 e22 e2 e77 e10 e5 e41 e1385 e9 b10 c251 d747 e39 e896 e0 e1 e7 e20 e16 e3 e6 e22 e2 e77 e10 e5 e1385 e9 e302'

In [20]:
Y[100]

0

In [47]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

In [110]:
# Классификатор Random Forest
def train_classifier(data, labels):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(data)
    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)
    
    random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    random_forest_model.fit(X_train, y_train)
    
    y_pred_rf = random_forest_model.predict(X_test)
    print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
    precision = precision_score(y_test, y_pred_rf)
    recall = recall_score(y_test, y_pred_rf)
    auc = roc_auc_score(y_test, y_pred_rf)
    # print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'AUC: {auc:.4f}')
    
    return random_forest_model, vectorizer

In [23]:
random_forest_model, vectorizer = train_classifier(X, Y)

Random Forest Accuracy: 0.7470133159894567
Precision: 0.7456
Recall: 0.7189
AUC: 0.7460


In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, random_state=42)
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)

In [62]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [63]:
# Метод наивный Байес
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [64]:
X_test_new = vectorizer.transform(X_test)
#print(X_test_new.shape)
X_new_tfidf = tfidf_transformer.transform(X_test_new)
predicted = clf.predict(X_new_tfidf)

In [65]:
kk = 0
for doc, category in zip(y_test, predicted):
    print(doc, category)
    if doc == category: kk +=1
        
print('model ', kk/len(y_test))

1 1
0 1
1 1
0 0
1 1
1 0
1 1
1 1
1 1
1 1
1 0
1 1
0 1
1 1
0 1
0 1
1 1
0 0
0 0
0 1
0 1
0 0
1 1
0 1
0 0
0 0
0 1
1 1
1 1
0 0
0 1
0 1
0 1
1 1
0 0
1 1
1 0
0 0
0 0
0 0
0 0
0 1
1 1
1 1
1 1
1 1
1 1
0 1
0 0
0 0
0 0
0 0
1 0
0 0
0 1
1 1
1 1
0 0
0 0
0 0
0 0
0 0
1 0
1 1
1 1
0 1
0 0
1 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
1 0
1 0
0 0
1 1
1 1
1 1
1 1
1 0
1 1
0 0
0 0
0 0
1 1
0 0
0 1
0 1
1 1
0 0
1 0
1 1
0 0
0 0
0 0
1 1
1 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 1
0 1
0 0
0 1
0 0
0 0
1 1
0 1
0 1
0 1
0 1
0 0
0 0
1 1
1 1
1 0
0 1
1 1
0 0
0 1
0 0
1 1
1 1
0 0
0 1
0 1
0 0
0 1
0 0
1 1
1 0
1 1
1 0
0 1
1 1
0 1
0 0
0 1
1 1
0 0
0 1
0 1
1 1
1 1
0 1
0 0
1 1
0 0
0 0
0 0
1 0
0 0
1 1
1 0
0 0
1 1
1 0
0 0
0 1
1 1
0 1
1 0
0 1
0 1
1 0
1 1
1 1
1 0
1 1
1 1
0 0
0 0
0 0
0 1
0 0
1 0
0 1
0 0
0 0
0 1
0 0
1 0
1 0
0 1
1 0
0 0
1 1
1 0
0 0
1 1
1 0
0 1
0 1
0 0
1 0
0 0
0 0
0 0
0 0
1 0
0 1
0 1
1 1
1 1
0 0
0 0
1 0
1 1
0 0
0 0
1 1
0 1
1 1
1 1
1 1
0 0
0 1
0 1
0 0
0 1
0 0
1 1
0 0
0 0
0 1
0 0
0 0
1 1
1 0
0 0
1 1
0 0
0 0
0 0
0 0
0 0
0 0
1 1
0 0
0 1
0 0
1 1
1 1


In [67]:
# Вывод метрик
accuracy = accuracy_score(y_test, predicted)
precision = precision_score(y_test, predicted)
recall = recall_score(y_test, predicted)
auc = roc_auc_score(y_test, predicted)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'AUC: {auc:.4f}')

Accuracy: 0.6741
Precision: 0.6512
Recall: 0.6979
AUC: 0.6750


#### Обработка текстов атрибутов

In [4]:
from pymystem3 import Mystem
import pickle
m = Mystem()

In [5]:
def get_lem_words(ss: str) -> list:
    '''
    Получение слов из строки
    '''
    ww= []
    ss = ss.strip().lower()
    ddel = ['*', ',', '-', '/', '%', ';', ')', '+', '.', '..', ':']
    for i in ddel:
        ss = ss.replace(i, ' ')
    ssl = ss.split()
    for word in ssl:
        # print(word)
        if not word.isdigit():
            lemmas = m.lemmatize(word)
            ww.append(lemmas[0])
    return ww
    

In [6]:
def get_all_text_catgory3(d_all, row, desc_all_text_cat3):
    '''
    Получение описание объекта в кодах-словах
    '''
    all_word = []
    dd = json.loads(row['categories'])
    # print(desc_all_text_cat4)
    # print(d_all['d4'][dd['4']])
    #[d_all)
    #      ['d4']])
    # print(desc_all_text_cat4[d_all['d4'][dd['4']]])
    # #print(row['characteristic_attributes_mapping'])
    if d_all['d3'][dd['3']] not in desc_all_text_cat3:
        desc_all_text_cat3[d_all['d3'][dd['3']]] = []
    ar = json.loads(row['characteristic_attributes_mapping'])
    #print(ar)
    for key in ar:
        if isinstance(ar[key], list):
            # print(key)
            for ww in ar[key]:
                
                all_word = all_word + get_lem_words(ww)
            # print()
    desc_all_text_cat3[d_all['d3'][dd['3']]] = desc_all_text_cat3[d_all['d3'][dd['3']]] + all_word
    return desc_all_text_cat3

In [16]:
desc_all_text_cat3 ={}
kk = 0
for i in range(0, len(attributes)):
    # print(attributes.iloc[i])
    desc_all_text_cat3 = get_all_text_catgory3(d_all, attributes.iloc[i], desc_all_text_cat3)
    # print(desc_all_text_cat3)
    # if kk == 3:
    #     break
    kk += 1
    if kk%100000 == 0:
        print(kk)

print(len(desc_all_text_cat3))
# print(desc_all_text_cat4)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
583


In [19]:
with open(PATH_DD + 'data/desc_all_text_cat3.pickle', 'bw') as f:
    pickle.dump(desc_all_text_cat3, f)
with open(PATH_DD + 'data/desc_all_text_cat3.pickle', 'br') as f:
    desc_all_text_cat3 = pickle.load(f)

In [20]:
sets_desc_all_text_cat3 ={}
for key in desc_all_text_cat3:
    if key not in sets_desc_all_text_cat3:
        sets_desc_all_text_cat3[key] = set(desc_all_text_cat3[key])
    # print(key, len(desc_all_text_cat4[key]))
    # print(len(set(desc_all_text_cat4[key])))
    # break

In [21]:
with open(PATH_DD + 'data/sets_desc_all_text_cat3.pickle', 'bw') as f:
    pickle.dump(sets_desc_all_text_cat3, f)
with open(PATH_DD + 'data/sets_desc_all_text_cat3.pickle', 'br') as f:
    sets_desc_all_text_cat3= pickle.load(f)

In [22]:
com_list_words = []
for key in sets_desc_all_text_cat3:
    com_list_words.append(sets_desc_all_text_cat3[key])
print(len(com_list_words))

583


In [23]:
int_result = set.intersection(*com_list_words)
len(int_result)

0

In [24]:
uni_result = set.union(*com_list_words)
len(uni_result)
    

992706

In [71]:
def get_diig(ss: str) -> str:
    '''
    удаление полностью чисел
    '''
    ddel = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    for i in ddel:
        ss = ss.replace(i, '')
    return ss

In [72]:
ssdd = set()
for ww in uni_result:
    ddd = get_diig(ww)
    if len(ddd) > 1:
        ssdd.add(ddd)
len(ssdd)

316005

In [76]:
with open(PATH_DD + 'data/ssdd.pickle', 'bw') as f:
    pickle.dump(ssdd, f)
with open(PATH_DD + 'data/ssdd.pickle', 'br') as f:
    ssdd = pickle.load(f)

In [25]:
set_com_words = set()
for key1 in sets_desc_all_text_cat3:
    for key2 in sets_desc_all_text_cat3:
        if key1 != key2:
            int_res = sets_desc_all_text_cat3[key1].intersection(sets_desc_all_text_cat3[key2])
            if len(int_res) > 0:
                for word in int_res:
                    set_com_words.add(word)
print(len(set_com_words))

106636


In [26]:
set_com_words

{'ports',
 'm20',
 '20х4',
 'апсо',
 'elitt',
 'clarette',
 's204',
 'гладить',
 't88',
 'этилфосфат',
 'senega',
 'tatiana',
 'l390',
 'стационарный',
 'vulcan',
 'лиминный',
 'shockwave',
 'урсоловый',
 'адаптоген',
 'кракль',
 'k81',
 '5crdi',
 'отжим',
 'багажный',
 '210х220',
 'фаберже',
 'titanate',
 'хлястик',
 'тигровый',
 'wife',
 '360х',
 'pauline',
 'октокрилен',
 'x550cl',
 'поликватерний',
 'ортосифон',
 'нерв',
 '85х85',
 '2na',
 'granite',
 '65d20',
 'abyssinica',
 'g8005',
 '13x13x13',
 'мухомор',
 'd35',
 'альтиметр',
 '39х19',
 'beijing',
 'дсб',
 '2мкг',
 'kmc',
 '10х10',
 'pcitr',
 'вседержитель',
 'kavkazsuvenir',
 'суперкомпактный',
 'd5000',
 'рогачев',
 'f2m5ns3wp',
 'ико',
 'x156',
 '3000х1600',
 'зил',
 'косс',
 'подарить',
 'табак',
 'родник',
 'соержать',
 'sku00120',
 'qsp',
 '210x70',
 'eu35',
 'xylinum',
 '14х10х10',
 'пейнтбол',
 'одинарный',
 '8gd',
 '5733z',
 'дезинсектицид',
 'конгресс',
 'сон',
 '50x150',
 'hd3',
 '150см',
 'falcone',
 'личностный',


In [27]:
with open(PATH_DD + 'data/set_com_words3.pickle', 'bw') as f:
    pickle.dump(set_com_words, f)
with open(PATH_DD + 'data/set_com_words.pickle', 'br') as f:
    set_com_words = pickle.load(f)

In [77]:
def get_lem_words_three(ss: str) -> list:
    '''
    Получение слов из строки
    '''
    ww= []
    ss = ss.strip().lower()
    ddel = ['*', ',', '-', '/', '%', ';', ')', '(', '[', ']','+', '.', '..', ':']
    for i in ddel:
        ss = ss.replace(i, ' ')
    ssl = ss.split()
    for word in ssl:
        # print(word)
        wwd = get_diig(word)
        if len(wwd) > 1:
            lemmas = m.lemmatize(wwd)
            ww.append(lemmas[0])
    return ww

In [28]:
def get_lem_words_two(ss: str) -> list:
    '''
    Получение слов из строки
    '''
    ww= []
    ss = ss.strip().lower()
    ddel = ['*', ',', '-', '/', '%', ';', ')', '+', '.', '..', ':']
    for i in ddel:
        ss = ss.replace(i, ' ')
    ssl = ss.split()
    for word in ssl:
        # print(word)
        if not word.isdigit():
            lemmas = m.lemmatize(word)
            if not lemmas[0] in set_com_words:
                ww.append(lemmas[0])
    return ww

In [30]:
def get_sequence_v(row):
    '''
    Получение предложения описания объекта в кодах-словах
    '''
    all_word = []
    ar = json.loads(row['characteristic_attributes_mapping'])
    #print(ar)
    for key in ar:
        if isinstance(ar[key], list):
            # print(key)
            for ww in ar[key]:
                
                all_word = all_word + get_lem_words_two(ww)
            # print()
    return ' '.join(all_word)

In [78]:
def get_sequence_three(row):
    '''
    Получение предложения описания объекта в кодах-словах
    '''
    all_word = []
    ar = json.loads(row['characteristic_attributes_mapping'])
    #print(ar)
    for key in ar:
        if isinstance(ar[key], list):
            # print(key)
            for ww in ar[key]:
                
                all_word = all_word + get_lem_words_three(ww)
            # print()
    return ' '.join(all_word)

In [85]:
#X_words_three = []
pusto = 0
Y_var_three = []
k = 0
for i in range(0, len(attributes)):
    v = attributes.iloc[i]['variantid']
    Y_var_three.append(str(v).strip())
    # sequence_three = get_sequence_three(attributes.iloc[i])
    # if sequence_three:
    #     X_words_three.append(sequence_three)
    # else:
    #     X_words_three.append(['pusto'])
    #     pusto += 1
    # if k == 10:
    #     break
    # k += 1
print(len(X_words_three))
print(len(X_words_three))
# print(X_words)

2252569
2252569


In [84]:
pusto

0

In [37]:
X_words = []
pusto = 0
Y_var = []
k = 0
for i in range(0, len(attributes)):
    v = attributes.iloc[i]['variantid']
    Y_var.append(str(v).strip())
    sequence_v = get_sequence_v(attributes.iloc[i])
    if sequence_v:
        X_words.append(sequence_v)
    else:
        X_words.append(['pusto'])
        pusto += 1
    # if k == 10:
    #     break
    # k += 1
print(len(X_words))
# print(X_words)

2252569


In [86]:
with open(PATH_DD + 'data/X_words_three.pickle', 'bw') as f:
     pickle.dump(X_words_three,f)
with open(PATH_DD + 'data/Y_var_three.pickle', 'bw') as f:
    pickle.dump(Y_var_three,f)
# with open(PATH_DD + 'data/X_words33.pickle', 'br') as f:
#     X_words = pickle.load(f)
# with open(PATH_DD + 'data/Y_var33.pickle', 'br') as f:
#     Y_var = pickle.load(f)

In [87]:
X_words_three[1000]

'проставка колесный шт комплектовать крепеж буклет информация товар буклет ассортимент магазин упаковка год maket проставка для колесный диск россия легковой автомобиль серебристый алюминий завод проставка zuz алюминий'

In [88]:
desc_all_obj4= {}
for key, sens in zip(Y_var_three, X_words_three):
    desc_all_obj4[key] = sens
desc_all_obj4

{'47920382': 'бежевый светло розовый унисекс funny ducks игрушка для ванна китай воображение кругозор моторика от год пвх пластизоль до год',
 '49801845': 'нет бренд страз акрил',
 '49853444': 'vervaco набор для вышивание бельгия акрил бумага канва металл хлопок простой крест от год',
 '49893028': 'серый prym тесьма ткань германия',
 '49987483': 'разноцветный мультиколор оригами настольный игра россия фабричный производство детская бумага картон пластик',
 '49995691': 'настольный игра китай башня пирамида детская бумага дерево разноцветный ласа играс для компания фабричный производство от год',
 '50101275': 'зеленый прозрачный полупрозрачный fimo глина полимерный германия',
 '50373843': 'синий нет бренд кабошон китай акрил',
 '51158084': 'ошо одеяло россия бежевый поликоттон шерсть belashoff стеганый фабричный производство',
 '51252362': 'чайник мл коробка картон чайник гунф kamjove китай механизм « фабричный производство стекло',
 '51435411': 'настольный офисный лампа китай накаливани

In [89]:
 with open(PATH_DD + 'data/desc_all_obj4.pickle', 'bw') as f:
    pickle.dump(desc_all_obj4, f)

In [43]:
label = []
X2=[]
Y2=[]
for row in train.iterrows():
    # print(row)
    # print(row[1]['variantid1'])
    # print(row[1]['variantid2'])
    # print(row[1]['target'])
    # label.append(row[0])
    if str(row[1]['variantid1']).strip() in desc_all_obj3 and str(row[1]['variantid2']).strip() in desc_all_obj3:
        
        # print(desc_all_obj[row[1]['variantid1']])
        a = ''
        b = ''
        if type(desc_all_obj3[str(row[1]['variantid1'])]) == list:
            a = 'pusto'
        else: a = desc_all_obj3[str(row[1]['variantid1']).strip()]
        if type(desc_all_obj3[str(row[1]['variantid2'])]) == list:
            b = 'pusto'
        else: b = desc_all_obj3[str(row[1]['variantid2']).strip()]
    X2.append(a + ' ' + b)
    Y2.append(row[1]['target'])
    # else:
    #     print('not found')
    #break
print(len(X2))

1168516


In [45]:
X2[1000]

'pusto pusto'

In [ ]:
vectorizer2, tfidf_transformer2, clf = train_classifierNB(X2, Y2)

In [31]:
random_forest_model2, vectorizer2 = train_classifier(X2, Y2)

: 

: 

In [61]:
label3 = []
X3=[]
Y3=[]
for row in train.iterrows():
    # print(row)
    # print(row[1]['variantid1'])
    # print(row[1]['variantid2'])
    # print(row[1]['target'])
    #label3.append(row[0])
    if str(row[1]['variantid1']).strip() in desc_all_obj and str(row[1]['variantid2']).strip() in desc_all_obj:
        # print(desc_all_obj[row[1]['variantid1']])
        a = ''
        b = ''
        if type(desc_all_obj3[str(row[1]['variantid1'])]) == list:
            a = 'pusto'
        else: a = desc_all_obj3[str(row[1]['variantid1']).strip()]
        if type(desc_all_obj3[str(row[1]['variantid2'])]) == list:
            b = 'pusto'
        else: b = desc_all_obj3[str(row[1]['variantid2']).strip()]
        X3.append(desc_all_obj[str(row[1]['variantid1']).strip()] + ' ' + a + ' ' +
                  desc_all_obj[str(row[1]['variantid2']).strip()] + ' ' + b)
        Y3.append(row[1]['target'])
    # else:
    #     print('not found')
    #break
print(len(X3))

1168516


In [62]:
with open(PATH_DD + 'data/X33.pickle', 'bw') as f:
    pickle.dump(X3, f)
with open(PATH_DD + 'data/Y33.pickle', 'bw') as f:
    pickle.dump(Y3, f)

In [74]:
def train_classifierNB(X3, Y3):
    X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, Y3, test_size=0.20, random_state=42)
    vectorizer3 = CountVectorizer()
    X_train_counts3 = vectorizer3.fit_transform(X_train3)
    tfidf_transformer3 = TfidfTransformer()
    X_train_tfidf3 = tfidf_transformer3.fit_transform(X_train_counts3)
    # Метод наивный Байес
    clf = MultinomialNB().fit(X_train_tfidf3, y_train3)
    X_test_new3 = vectorizer3.transform(X_test3)
    #print(X_test_new.shape)
    X_new_tfidf3 = tfidf_transformer3.transform(X_test_new3)
    predicted3 = clf.predict(X_new_tfidf3)
    # kk = 0
    # for doc, category in zip(y_test3, predicted3):
    #     print(doc, category)
    #     if doc == category: kk +=1
            
    # print('model ', kk/len(y_test3))
    # Вывод метрик
    accuracy3 = accuracy_score(y_test3, predicted3)
    precision3 = precision_score(y_test3, predicted3)
    recall3 = recall_score(y_test3, predicted3)
    auc3 = roc_auc_score(y_test3, predicted3)
    
    print(f'Accuracy: {accuracy3:.4f}')
    print(f'Precision: {precision3:.4f}')
    print(f'Recall: {recall3:.4f}')
    print(f'AUC: {auc3:.4f}')

    return vectorizer3, tfidf_transformer3, clf

In [75]:
vectorizer3, tfidf_transformer3, clf = train_classifierNB(X3, Y3)

Accuracy: 0.6810
Precision: 0.6605
Recall: 0.6921
AUC: 0.6814


In [70]:
random_forest_model3, vectorizer3 = train_classifier(X3, Y3)

Random Forest Accuracy: 0.7292874038524076
Precision: 0.7156
Recall: 0.7253
AUC: 0.7291


In [90]:
label4 = []
X4=[]
Y4=[]
for row in train.iterrows():
    # print(row)
    # print(row[1]['variantid1'])
    # print(row[1]['variantid2'])
    # print(row[1]['target'])
    #label3.append(row[0])
    if str(row[1]['variantid1']).strip() in desc_all_obj and str(row[1]['variantid2']).strip() in desc_all_obj:
        # print(desc_all_obj[row[1]['variantid1']])
        a = ''
        b = ''
        if type(desc_all_obj4[str(row[1]['variantid1'])]) == list:
            a = 'pusto'
        else: a = desc_all_obj4[str(row[1]['variantid1']).strip()]
        if type(desc_all_obj4[str(row[1]['variantid2'])]) == list:
            b = 'pusto'
        else: b = desc_all_obj4[str(row[1]['variantid2']).strip()]
        X4.append(desc_all_obj[str(row[1]['variantid1']).strip()] + ' ' + a + ' ' +
                  desc_all_obj[str(row[1]['variantid2']).strip()] + ' ' + b)
        Y4.append(row[1]['target'])
    # else:
    #     print('not found')
    #break
print(len(X4))

1168516


In [92]:
with open(PATH_DD + 'data/X4.pickle', 'bw') as f:
    pickle.dump(X4, f)
with open(PATH_DD + 'data/Y4.pickle', 'bw') as f:
    pickle.dump(Y4, f)

In [93]:
vectorizer4, tfidf_transformer4, clf = train_classifierNB(X4, Y4)

Accuracy: 0.7065
Precision: 0.6902
Recall: 0.7065
AUC: 0.7065


In [111]:
random_forest_model4, vectorizer4 = train_classifier(X4, Y4)

Random Forest Accuracy: 0.7838419539246226
Precision: 0.7728
Recall: 0.7794
AUC: 0.7837
